In [43]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from jubileedemo import meatspace_protocols as meat
from jubileedemo import user_functions
import numpy as np
from bayesopt import acquisitions 
import json

import cv2

import bayesopt.bayesian_optimizer as bayesian_optimizer
import bayesopt.model as model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels

In [20]:
from jubileedemo import image_processing as img

In [21]:
jub = meat.BayesianOptDemoDriver(deck_config_filepath = 'deck_config.json')

Loading deck config file from deck_config.json


In [22]:
jub.safe_z = None

# Get 12 points of random data

In [20]:
sample_space = user_functions.get_constrained_points(101)

In [21]:
n_init_points = 12
sampled_indices = np.random.randint(0, len(sample_space), size = n_init_points)
sampled_points = sample_space[sampled_indices]

In [22]:
rgb_vals = []
for i in range(len(sampled_points)):
    point = sampled_points[i,:]
    well = jub.next_sample_well()
    print('creating sample')
    print(point)
    print(well)
    
    RGB = user_functions.sample_point(jub, point, 5, well)
    rgb_vals.append(RGB)

creating sample
[0.37 0.49 0.14]
3.A1
Dispensing volumes:  [1.85, 2.45, 0.7000000000000001]
well image function
current active tool:  1
creating sample
[0.72 0.14 0.14]
3.A2
Dispensing volumes:  [3.5999999999999996, 0.7000000000000001, 0.7000000000000001]
well image function
current active tool:  1
creating sample
[0.35 0.61 0.04]
3.A3
Dispensing volumes:  [1.7500000000000002, 3.05, 0.2]
well image function
current active tool:  1
creating sample
[0.55 0.2  0.25]
3.A4
Dispensing volumes:  [2.75, 1.0, 1.25]
well image function
current active tool:  1
creating sample
[0.27 0.33 0.4 ]
3.B1
Dispensing volumes:  [1.35, 1.6500000000000001, 2.0]
well image function
current active tool:  1
creating sample
[0.25 0.25 0.5 ]
3.B2
Dispensing volumes:  [1.25, 1.25, 2.5]
well image function
current active tool:  1
creating sample
[0.26 0.12 0.62]
3.B3
Dispensing volumes:  [1.3, 0.6, 3.1]
well image function
current active tool:  1
creating sample
[0.17 0.22 0.61]
3.B4
Dispensing volumes:  [0.8500000

In [34]:
loss_vals = [user_functions.color_loss_calculation(target_color, user_functions.normalize_color(rgb)) for rgb in rgb_vals]

In [44]:
init_data = []
for i in range(len(sampled_points)):
    data = {}
    data['sample_id'] = str(i)
    data['RYB_point'] = list(sampled_points[i])
    data['RGB_measured'] = rgb_vals[i]

    init_data.append(data)
    

In [45]:
with open('initial_data_random_12.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

# BO campaign

In [25]:
target_color = (178, 255, 102) #peachy
target_color = user_functions.normalize_color(target_color)

In [26]:
init_data = []

with open('initial_data_random_12.jsonl', 'rt') as f:
    for line in f:
        init_data.append(json.loads(line))

In [27]:
init_X = np.zeros((len(init_data),3))
init_y = np.zeros(len(init_data))

for i, entry in enumerate(init_data):
    init_X[i,:] = entry['RYB_point']
    init_y[i] = user_functions.color_loss_calculation(target_color, user_functions.normalize_color(entry['RGB_measured']))
    

In [28]:
acquisition_function = acquisitions.optimize_EI

In [31]:
initial_data = (init_X, init_y)
number_of_iterations = 21
jubilee = jub

In [32]:
bo_obj, ryb_sampled, rgb_measured = user_functions.BO_campaign(initial_data, acquisition_function, number_of_iterations, target_color, jubilee)

first query pt:  [0. 1. 0.]
Starting iteration 0
Dispensing into well 3.A4
RYB values tested: [0. 1. 0.]
RYB after norm in sample func:  [0.0, 1.0, 0.0]
Dispensing volumes:  [0.0, 5.0, 0.0]
current active tool:  1
swapping
loss score: 0.5447787628585046
RGB values observed: [0.8328626951099114, 0.7222109748934629, 0.06551194091403291]
Starting iteration 1
Dispensing into well 3.B1
RYB values tested: [0. 0. 1.]
RYB after norm in sample func:  [0.0, 0.0, 1.0]
Dispensing volumes:  [0.0, 0.0, 5.0]
current active tool:  1
swapping
loss score: 0.10501684450842363
RGB values observed: [0.0073154719879107676, 0.5567192791711969, 0.7569282295454829]
Starting iteration 2
Dispensing into well 3.B2
RYB values tested: [1. 0. 0.]
RYB after norm in sample func:  [1.0, 0.0, 0.0]
Dispensing volumes:  [5.0, 0.0, 0.0]
current active tool:  1
swapping
loss score: 0.24923765393396868
RGB values observed: [0.9714727752158863, 0.30239068221297444, 0.4471108111823442]
Starting iteration 3
Dispensing into well

In [7]:
import pickle

In [33]:
with open(

In [28]:
for i in range(12, 24):
    point = {}
    point['sample_id'] = str(i)
    point['RYB_point'] = list(ryb_points_sampled[i-12])
    point['RGB_measured'] = rgb_values_sampled[i-12]
    init_data.append(point)

In [29]:
init_data

[{'sample_id': '0',
  'RYB_point': [0.37, 0.49, 0.14],
  'RGB_measured': [101.51293817718292,
   103.28718929254302,
   172.56010197578075]},
 {'sample_id': '1',
  'RYB_point': [0.72, 0.14, 0.14],
  'RGB_measured': [133.88859145952836, 95.62651370299554, 181.11612492033143]},
 {'sample_id': '2',
  'RYB_point': [0.35000000000000003, 0.61, 0.04],
  'RGB_measured': [115.27877629063097, 136.3812619502868, 200.83671128107073]},
 {'sample_id': '3',
  'RYB_point': [0.55, 0.2, 0.25],
  'RGB_measured': [125.35436583811345, 92.0970044614404, 144.84537922243467]},
 {'sample_id': '4',
  'RYB_point': [0.27, 0.33, 0.4],
  'RGB_measured': [128.74225621414914,
   108.36583811344805,
   151.20038240917782]},
 {'sample_id': '5',
  'RYB_point': [0.25, 0.25, 0.5],
  'RGB_measured': [184.46513702995537, 166.2295729764181, 126.10325047801147]},
 {'sample_id': '6',
  'RYB_point': [0.26, 0.12, 0.62],
  'RGB_measured': [183.86271510516252,
   148.26602931803697,
   114.18891013384321]},
 {'sample_id': '7',
  '

In [31]:
with open('initial_data_24pts_combine.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

In [65]:
#image = cv2.imread('sampleimage_masked_1687813525.8915021.jpg')
image = cv2.imread('sampleimage_masked_1687813298.037668.jpg')

In [71]:
from jubileedemo import image_processing

In [72]:
rgb = image_processing._get_rgb_avg(image)

swapping


In [73]:
norm_rgb = user_functions.normalize_color(rgb)

In [74]:
user_functions.color_loss_calculation(target_color, norm_rgb)

0.2617500255474996

In [75]:
norm_rgb

[0.6621086280218055, 0.19737187036334725, 0.28539823707644374]

In [76]:
target_color

[0.5333333333333333, 0.011764705882352941, 0.9882352941176471]

In [67]:
cv2.imshow('test', image)
cv2.waitKey(0)   #wait for a keyboard input
cv2.destroyAllWindows()

In [68]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [34]:
with open('sample_rgb_values.pkl', 'rb') as f:
    test_rgb = pickle.load(f)

In [37]:
test_rgb

[array([0., 1., 0.]),
 array([0., 0., 1.]),
 array([1., 0., 0.]),
 array([0.45, 0.  , 0.55]),
 array([0.  , 0.76, 0.24]),
 array([0.27, 0.18, 0.55]),
 array([0.16, 0.84, 0.  ]),
 array([0.22, 0.78, 0.  ]),
 array([0.11, 0.89, 0.  ]),
 array([0.  , 0.34, 0.66]),
 array([0.16, 0.77, 0.07]),
 array([0.6, 0.4, 0. ]),
 array([0.84, 0.16, 0.  ]),
 array([0.27, 0.  , 0.73]),
 array([0.73, 0.  , 0.27]),
 array([0.13, 0.54, 0.33]),
 array([0.27, 0.73, 0.  ]),
 array([0.19, 0.81, 0.  ]),
 array([0.4 , 0.21, 0.39]),
 array([0.88, 0.  , 0.12]),
 array([0.  , 0.58, 0.42]),
 array([0.4 , 0.33, 0.27]),
 array([0.48, 0.52, 0.  ]),
 array([0. , 0.9, 0.1])]

In [40]:
loss_vals = []
for entry in test_rgb:
    loss = user_functions.color_loss_calculation(entry, target_color)
    loss_vals.append(loss)
    

In [41]:
import matplotlib.pyplot as plt

In [44]:
plt.plot(loss_vals)